In [14]:
import pandas as pd
import optuna
from lightgbm import LGBMClassifier
from sklearn.metrics import mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split
import requests
import json
import datetime
import copy

In [15]:
from pathlib import Path
path = Path.cwd().parent
raw_data = str(path) + '/raw_data/'
data = str(path) + '/data/'

# 데이터 Load

In [23]:
df = pd.read_csv(data +'dataset.csv')
df.drop('Unnamed: 0',axis =1,inplace= True)
# df.reset_index(inplace=True)
predict_df = pd.read_csv(data +'predict_dataset.csv')
predict_df.drop('Unnamed: 0',axis =1,inplace= True)


In [18]:
# # 일자별로 train_test 나누기
# train_data = df[df['일']<=25]
# test_data = df[df['일']>25]


#랜덤하게 나누기
train_data, test_data = train_test_split( df, test_size=0.33, random_state=42)

In [21]:
X_train , y_train = train_data.iloc[: , 0:14], train_data['혼잡도']
X_test , y_test = test_data.iloc[: , 0:14], test_data['혼잡도']

In [22]:
model = LGBMClassifier()
model.fit(X_train, y_train, categorical_feature =['주차장'], eval_set =(X_test,y_test) , verbose=1)

/Users/keumhyeonjun/opt/anaconda3/envs/행안부/lib/python3.9/site-packages/lightgbm/basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
/Users/keumhyeonjun/opt/anaconda3/envs/행안부/lib/python3.9/site-packages/lightgbm/basic.py:1554: UserWarning: categorical_feature in Dataset is overridden.
New categorical_feature is ['주차장']
  warnings.warn('categorical_feature in Dataset is overridden.\n'
/Users/keumhyeonjun/opt/anaconda3/envs/행안부/lib/python3.9/site-packages/lightgbm/basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
/Users/keumhyeonjun/opt/anaconda3/envs/행안부/lib/python3.9/site-packages/lightgbm/basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


[1]	valid_0's multi_logloss: 1.18691
[2]	valid_0's multi_logloss: 1.04294
[3]	valid_0's multi_logloss: 0.932233
[4]	valid_0's multi_logloss: 0.840291
[5]	valid_0's multi_logloss: 0.765892
[6]	valid_0's multi_logloss: 0.702706
[7]	valid_0's multi_logloss: 0.649711
[8]	valid_0's multi_logloss: 0.598
[9]	valid_0's multi_logloss: 0.555547
[10]	valid_0's multi_logloss: 0.518208
[11]	valid_0's multi_logloss: 0.484913
[12]	valid_0's multi_logloss: 0.455298
[13]	valid_0's multi_logloss: 0.428746
[14]	valid_0's multi_logloss: 0.405954
[15]	valid_0's multi_logloss: 0.383777
[16]	valid_0's multi_logloss: 0.364917
[17]	valid_0's multi_logloss: 0.346509
[18]	valid_0's multi_logloss: 0.329968
[19]	valid_0's multi_logloss: 0.316491
[20]	valid_0's multi_logloss: 0.303603
[21]	valid_0's multi_logloss: 0.292013
[22]	valid_0's multi_logloss: 0.280831
[23]	valid_0's multi_logloss: 0.271397
[24]	valid_0's multi_logloss: 0.262039
[25]	valid_0's multi_logloss: 0.254543
[26]	valid_0's multi_logloss: 0.246265


LGBMClassifier()

# 예측

In [35]:
predict = pd.DataFrame(model.predict(predict_df))
output = pd.concat([predict_df, predict], axis= 1)
#output = output.rename({'0':'혼잡도예측'})
output = output.rename(columns={0:'혼잡도예측'})
output = output[['연','월','일','시','분','요일','주차장','혼잡도예측']]
output.to_csv('주차예측결과.csv')